In [ ]:
%load_ext autoreload
import os
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt

%autoreload 2
from zcu_tools import load_data
from zcu_tools.analysis.fluxdep import (
    InteractiveLines,
    InteractiveFindPoints,
    InteractiveSelector,
    VisualizeSpet,
    calculate_energy,
    remove_close_points,
    preprocess_data,
    search_in_database,
    fit_spectrum,
    dump_result,
    load_result,
)

In [ ]:
qub_name = "Sapphire150"

mA_c = None
mA_e = None
period = None
s_spects = {}

os.makedirs(f"../result/{qub_name}/image", exist_ok=True)
os.makedirs(f"../result/{qub_name}/web", exist_ok=True)

In [ ]:
loadpath = f"../result/{qub_name}/params.json"
_, sp_params, mA_c, period, allows = load_result(loadpath)

mA_e = mA_c + 0.5 * period

# Load Spectrum

In [ ]:
spect_path = "../Database/Sapphire150/Sapphire150_TwoTone_flux_007.hdf5"
spectrum, fpts, mAs = load_data(spect_path, server_ip="100.105.52.95", port=4999)
mAs, fpts, spectrum = preprocess_data(mAs, fpts, spectrum)

s_spectrum = np.abs(spectrum - np.mean(spectrum, axis=0, keepdims=True))
s_spectrum /= np.std(s_spectrum, axis=0, keepdims=True)

In [ ]:
%matplotlib widget
actLine = InteractiveLines(s_spectrum, mAs, fpts, mA_c, mA_e)

In [ ]:
mA_c, mA_e = actLine.get_positions()
period = 2 * abs(mA_e - mA_c)

mA_c, mA_e, period

In [ ]:
%matplotlib widget
actSel = InteractiveFindPoints(spectrum, mAs, fpts, threshold=5.0)

In [ ]:
ss_mAs, ss_fpts = actSel.get_positions()

In [ ]:
name = spect_path.split("/")[-1]
s_spects.update(
    {
        name: {
            "mA_c": mA_c,
            "period": period,
            "spectrum": {
                "mAs": mAs,
                "fpts": fpts,
                "data": spectrum,
            },
            "points": {
                "mAs": ss_mAs,
                "fpts": ss_fpts,
            },
        }
    }
)
s_spects.keys()

# Save & Load

In [ ]:
# for val in s_spects.values():  # swap mA_c and mA_e
#     val["mA_c"] = mA_c + 0.5 * period

In [ ]:
save_path = f"../result/{qub_name}/spect.hdf5"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with h5.File(save_path, "x") as f:
    for path, val in s_spects.items():
        grp = f.create_group(path)
        grp.create_dataset("mA_c", data=val["mA_c"])
        grp.create_dataset("period", data=val["period"])
        spect_grp = grp.create_group("spectrum")
        spect_grp.create_dataset("mAs", data=val["spectrum"]["mAs"])
        spect_grp.create_dataset("fpts", data=val["spectrum"]["fpts"])
        spect_grp.create_dataset("data", data=val["spectrum"]["data"])
        points_grp = grp.create_group("points")
        points_grp.create_dataset("mAs", data=val["points"]["mAs"])
        points_grp.create_dataset("fpts", data=val["points"]["fpts"])

In [ ]:
load_path = f"../result/{qub_name}/spect.hdf5"
s_spects = {}
with h5.File(load_path, "r") as f:
    for key in f.keys():
        grp = f[key]
        _mA_c = grp["mA_c"][()]
        _period = grp["period"][()]
        s_spects.update(
            {
                key: {
                    "mA_c": _mA_c,
                    "period": _period,
                    "spectrum": {
                        "mAs": grp["spectrum"]["mAs"][()],
                        "fpts": grp["spectrum"]["fpts"][()],
                        "data": grp["spectrum"]["data"][()],
                    },
                    "points": {
                        "mAs": grp["points"]["mAs"][()],
                        "fpts": grp["points"]["fpts"][()],
                    },
                }
            }
        )
s_spects.keys()

# Align half flux

In [ ]:
mA_c = list(s_spects.values())[-1]["mA_c"]
period = list(s_spects.values())[-1]["period"]
for spect in s_spects.values():
    shift = mA_c - spect["mA_c"]
    spect["mA_c"] += shift
    spect["spectrum"]["mAs"] += shift
    spect["points"]["mAs"] += shift

In [ ]:
mA_bound = (
    np.nanmin([np.nanmin(s["spectrum"]["mAs"]) for s in s_spects.values()]),
    np.nanmax([np.nanmax(s["spectrum"]["mAs"]) for s in s_spects.values()]),
)
fpt_bound = (
    np.nanmin([np.nanmin(s["spectrum"]["fpts"]) for s in s_spects.values()]),
    np.nanmax([np.nanmax(s["spectrum"]["fpts"]) for s in s_spects.values()]),
)
sp_selected = None
t_mAs = np.linspace(mA_bound[0], mA_bound[1], 1000)
t_fpts = np.linspace(fpt_bound[0], fpt_bound[1], 1000)
t_flxs = (t_mAs - mA_c) / period

# Manual Remove Points

In [ ]:
%matplotlib widget
intSel = InteractiveSelector(s_spects, selected=sp_selected)

In [ ]:
sp_flxs, sp_fpts, sp_selected = intSel.get_positions()

In [ ]:
# # load selected data
# loadpath = f"../result/{qub_name}/selected.npz"
# with np.load(loadpath) as data:
#     sp_flxs = data["flxs"]
#     sp_fpts = data["fpts"]
#     sp_selected = data["selected"]

In [ ]:
s_flxs, s_fpts = remove_close_points(sp_flxs, sp_fpts, min_fpt_dist=10e-3)

print(s_flxs.shape, sp_fpts.shape)

fig, ax = plt.subplots()
ax.scatter(s_flxs, s_fpts, s=1)
plt.show()

# Fitting range

In [ ]:
# EJb = (1.0, 6.0)
# ECb = (0.8, 2.0)
# ELb = (0.01, 0.2)
# EJb = (3.0, 15.0)
# ECb = (0.2, 2.0)
# ELb = (0.5, 2.0)
EJb = (1.0, 20.0)
ECb = (0.1, 4.0)
ELb = (0.01, 3.0)

# Search in Database

In [ ]:
allows = {
    "transitions": [(0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (1, 6), (2, 7)],
}
allows = {
    **allows,
    # "transitions": [(i, j) for i in (0, 1, 2) for j in range(5) if i < j],
    # "red side": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "blue side": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "mirror": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "transitions2": [(i, j) for i in (0, 1, 2) for j in range(5) if i < j],
    # "red side2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "blue side2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
    # "mirror2": [(i, j) for i in (0, 1, 2) for j in range(8) if i < j],
}

In [ ]:
best_params, fig = search_in_database(
    s_flxs, s_fpts, "../Database/simulation/fluxonium_all.h5", allows, EJb, ECb, ELb
)
fig.savefig(f"../result/{qub_name}/image/search_result.png")

In [ ]:
energies = calculate_energy(t_flxs, *best_params, cutoff=40, evals_count=15)

In [ ]:
v_allows = {
    **allows,
    "transitions": [(i, j) for i in (0, 1, 2) for j in range(i + 1, 15)],
    "transitions2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "blue side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "blue side2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "red side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "red side2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
}

vs = VisualizeSpet(
    s_spects, s_flxs, s_fpts, t_flxs, energies, v_allows, auto_hide=False
)
fig = vs.create_figure()
fig.update_layout(
    title=f"EJ/EC/EL = ({best_params[0]:.2f}, {best_params[1]:.2f}, {best_params[2]:.2f})",
    title_x=0.5,
)
fig.show()

# Scipy Optimization

In [ ]:
# fit the spectrumData
sp_params = fit_spectrum(s_flxs, s_fpts, best_params, allows, (EJb, ECb, ELb))

# print the results
print("Fitted params:", *sp_params)

In [ ]:
energies = calculate_energy(t_flxs, *sp_params, 40, 15)

In [ ]:
v_allows = {
    **allows,
    "transitions": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    "transitions2": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "blue side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
    # "red side": [(i, j) for i in [0, 1, 2] for j in range(i + 1, 15)],
}

vs = VisualizeSpet(s_spects, s_flxs, s_fpts, t_flxs, energies, v_allows, auto_hide=True)
fig = vs.create_figure()
fig.update_layout(
    title=f"EJ/EC/EL = ({sp_params[0]:.4f}, {sp_params[1]:.4f}, {sp_params[2]:.4f})",
    title_x=0.501,
)
fig.show()

# Save Parameters

In [ ]:
fig.write_html(f"../result/{qub_name}/web/spect_fit.html", include_plotlyjs="cdn")
fig.write_image(f"../result/{qub_name}/image/spect_fit.png", format="png")

In [ ]:
# dump the data
savepath = f"../result/{qub_name}/params.json"

dump_result(savepath, qub_name, sp_params, mA_c, period, allows)

In [ ]:
savepath = f"../result/{qub_name}/selected.npz"

np.savez(savepath, flxs=sp_flxs, fpts=sp_fpts, selected=sp_selected)